# Resultados

## imports

In [17]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [18]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index'])
print("Dataframe shape: ", df.shape)
df.head()
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/test.csv")
test = test.drop(columns=['index'])

Dataframe shape:  (1400, 27)


In [19]:
y = df['label']
X = df.drop(columns=['label'])
y = y.to_numpy()
X = X.to_numpy()    
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

## Datos completos

In [20]:
df_results = pd.read_csv('results2.csv')
df_results = df_results.sort_values(by='accuracy', ascending=False)
df_results.head(5)

,lr,bs,mdl,nr,accuracy,f1_micro,f1_macro
5,0.0001,2000,1.000000e-06,7,0.6775,0.6775,0.673366
26,0.0001,4000,1.000000e-05,7,0.6675,0.6675,0.666748
16,0.0001,3000,1.000000e-05,5,0.6650,0.6650,0.664790
10,0.0001,3000,1.000000e-07,5,0.6650,0.6650,0.662264
4,0.0001,2000,1.000000e-06,5,0.6600,0.6600,0.659864


In [21]:
DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=0.0001, batch_size=2000,
                                lossfn="MSE", num_workers=1, min_dloss=1e-6, precompute_rules=True)

losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=7, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
y_pred = DSC.predict(X_test)


Optimization started
Processing epoch	500	0.2110	
Training time: 1236.51s, epochs: 500

Least training loss reached: 0.211


In [22]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy:  0.64
[[105  83]
 [ 61 151]]
              precision    recall  f1-score   support

           0       0.63      0.56      0.59       188
           1       0.65      0.71      0.68       212

    accuracy                           0.64       400
   macro avg       0.64      0.64      0.64       400
weighted avg       0.64      0.64      0.64       400



In [23]:
DSC.print_most_important_rules(classes=["0",  "1"])



Most important rules for class 0

	[0.236] R40: b6(Y) < 0.441
			0: 0.236	1: 0.000	Unc: 0.764

	[0.227] R94: 0.960 < b10(Y) < 1.137
			0: 0.227	1: 0.000	Unc: 0.773

	[0.217] R60: 0.669 < b9(Y) < 0.800
			0: 0.217	1: 0.000	Unc: 0.783

	[0.210] R170: -0.014 < b12(Y) < 0.124
			0: 0.210	1: 0.000	Unc: 0.790

	[0.205] R138: 0.717 < b1(Y) < 0.806
			0: 0.205	1: 0.000	Unc: 0.795

Most important rules for class 1

	[0.208] R106: 0.276 < b8(Y) < 0.421
			0: 0.000	1: 0.208	Unc: 0.792

	[0.208] R45: 0.930 < b6(Y) < 1.048
			0: 0.000	1: 0.208	Unc: 0.792

	[0.205] R13: 0.975 < b2(Y) < 1.067
			0: 0.000	1: 0.205	Unc: 0.795


## Datos postivos

### pre procesado

In [24]:
df_pos = df.drop(columns=["b1(N)", "b2(N)", "b3(N)", "b4(N)", "b5(N)", "b6(N)", "b7(N)", "b8(N)", "b9(N)", "b10(N)", "b11(N)", "b12(N)", "b13(N)"])
test_pos = test.drop(columns=["b1(N)", "b2(N)", "b3(N)", "b4(N)", "b5(N)", "b6(N)", "b7(N)", "b8(N)", "b9(N)", "b10(N)", "b11(N)", "b12(N)", "b13(N)"])
y_pos = df_pos['label']
X_pos = df_pos.drop(columns=['label'])
y_pos = y_pos.to_numpy()
X_pos = X_pos.to_numpy()
y_test_pos = test_pos['label']
X_test_pos = test_pos.drop(columns=['label'])
y_test_pos = y_test_pos.to_numpy()
X_test_pos = X_test_pos.to_numpy()

### resultados

In [25]:
df_results_pos = pd.read_csv('results_pos.csv')
df_results_pos = df_results_pos.sort_values(by='accuracy', ascending=False)
df_results_pos.head(5)

,lr,bs,mdl,nr,accuracy,f1_micro,f1_macro
11,0.0001,3000,1.000000e-07,7,0.7000,0.7000,0.697268
21,0.0001,4000,1.000000e-06,3,0.6825,0.6825,0.680050
10,0.0001,3000,1.000000e-07,5,0.6800,0.6800,0.677939
19,0.0001,4000,1.000000e-07,5,0.6775,0.6775,0.676024
25,0.0001,4000,1.000000e-05,5,0.6775,0.6775,0.677256


In [26]:
DSC_pos = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=0.0001, batch_size=3000,
                                lossfn="MSE", num_workers=1, min_dloss=1e-7, precompute_rules=True)

losses_pos, epoch_pos, dt_pos = DSC_pos.fit(X_pos, y_pos, add_single_rules=True,
                            single_rules_breaks=7, add_mult_rules=False,
                                column_names=df_pos.columns[:-1], print_every_epochs=1, print_final_model=False)

y_pred_pos = DSC_pos.predict(X_test_pos)

Optimization started
Processing epoch	1	1.0000	

Processing epoch	500	0.2183	
Training time: 1251.79s, epochs: 500

Least training loss reached: 0.218


In [27]:
acc_pos = accuracy_score(y_test_pos, y_pred_pos)
print("Accuracy: ", acc_pos)
print(confusion_matrix(y_test_pos, y_pred_pos))
print(classification_report(y_test_pos, y_pred_pos))

Accuracy:  0.6675
[[106  82]
 [ 51 161]]
              precision    recall  f1-score   support

           0       0.68      0.56      0.61       188
           1       0.66      0.76      0.71       212

    accuracy                           0.67       400
   macro avg       0.67      0.66      0.66       400
weighted avg       0.67      0.67      0.66       400



In [40]:
DSC_pos.print_most_important_rules(classes=["0",  "1"], threshold=0.19)



Most important rules for class 0

	[0.228] R46: 0.960 < b10(Y) < 1.137
			0: 0.228	1: 0.000	Unc: 0.772

	[0.227] R17: 0.441 < b6(Y) < 0.600
			0: 0.227	1: 0.000	Unc: 0.773

	[0.217] R89: -0.155 < b11(Y) < 0.044
			0: 0.217	1: 0.000	Unc: 0.783

	[0.216] R20: 0.824 < b6(Y) < 0.930
			0: 0.216	1: 0.000	Unc: 0.784

	[0.215] R8: b7(Y) < 0.611
			0: 0.215	1: 0.000	Unc: 0.785

	[0.203] R65: 0.597 < b1(Y) < 0.717
			0: 0.203	1: 0.000	Unc: 0.797

	[0.193] R37: 1.014 < b5(Y) < 1.050
			0: 0.186	1: 0.014	Unc: 0.800

	[0.192] R73: 0.255 < b3(Y) < 0.438
			0: 0.186	1: 0.014	Unc: 0.801

	[0.190] R100: 0.299 < b13(Y) < 0.430
			0: 0.181	1: 0.018	Unc: 0.800

Most important rules for class 1

	[0.223] R69: 0.967 < b1(Y) < 1.056
			0: 0.000	1: 0.223	Unc: 0.777

	[0.196] R55: b8(Y) > 1.022
			0: 0.008	1: 0.192	Unc: 0.800

	[0.196] R79: b3(Y) > 1.141
			0: 0.009	1: 0.191	Unc: 0.800

	[0.191] R13: 0.932 < b7(Y) < 1.009
			0: 0.014	1: 0.184	Unc: 0.803


## Datos negativos

### Preprocesado

In [29]:
df_neg = df.drop(columns=["b1(Y)", "b2(Y)", "b3(Y)", "b4(Y)", "b5(Y)", "b6(Y)", "b7(Y)", "b8(Y)", "b9(Y)", "b10(Y)", "b11(Y)", "b12(Y)", "b13(Y)"])
test_neg = test.drop(columns=["b1(Y)", "b2(Y)", "b3(Y)", "b4(Y)", "b5(Y)", "b6(Y)", "b7(Y)", "b8(Y)", "b9(Y)", "b10(Y)", "b11(Y)", "b12(Y)", "b13(Y)"])
y_neg = df_neg['label']
X_neg = df_neg.drop(columns=['label'])
y_neg = y_neg.to_numpy()
X_neg = X_neg.to_numpy()
y_test_neg = test_neg['label']
X_test_neg = test_neg.drop(columns=['label'])
y_test_neg = y_test_neg.to_numpy()
X_test_neg = X_test_neg.to_numpy()

### Resultados

In [30]:
df_results_neg = pd.read_csv('results_neg.csv')
df_results_neg = df_results_neg.sort_values(by='accuracy', ascending=False)
df_results_neg.head(5)

,lr,bs,mdl,nr,accuracy,f1_micro,f1_macro
16,0.0001,3000,1.000000e-05,5,0.6900,0.6900,0.689930
13,0.0001,3000,1.000000e-06,5,0.6800,0.6800,0.679800
9,0.0001,3000,1.000000e-07,3,0.6750,0.6750,0.673400
6,0.0001,2000,1.000000e-05,3,0.6700,0.6700,0.667874
12,0.0001,3000,1.000000e-06,3,0.6675,0.6675,0.665491


In [31]:
DSC_neg = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=0.0001, batch_size=3000,
                                lossfn="MSE", num_workers=1, min_dloss=1e-5, precompute_rules=True)

losses_neg, epoch_neg, dt_neg = DSC_neg.fit(X_neg, y_neg, add_single_rules=True,
                            single_rules_breaks=5, add_mult_rules=False,
                                column_names=df_neg.columns[:-1], print_every_epochs=1, print_final_model=False)

y_pred_neg = DSC_neg.predict(X_test_neg)

Optimization started
Processing epoch	1	1.0000	

Processing epoch	500	0.2176	
Training time: 1344.71s, epochs: 500

Least training loss reached: 0.218


In [32]:
acc_neg = accuracy_score(y_test_neg, y_pred_neg)
print("Accuracy: ", acc_neg)
print(confusion_matrix(y_test_neg, y_pred_neg))
print(classification_report(y_test_neg, y_pred_neg))

Accuracy:  0.6725
[[126  62]
 [ 69 143]]
              precision    recall  f1-score   support

           0       0.65      0.67      0.66       188
           1       0.70      0.67      0.69       212

    accuracy                           0.67       400
   macro avg       0.67      0.67      0.67       400
weighted avg       0.67      0.67      0.67       400



In [41]:
DSC_neg.print_most_important_rules(classes=["0",  "1"], threshold=0.19)



Most important rules for class 0

	[0.210] R59: b3(N) > 0.651
			0: 0.210	1: 0.000	Unc: 0.790

	[0.209] R70: 0.865 < b11(N) < 1.084
			0: 0.209	1: 0.000	Unc: 0.791

	[0.201] R57: 0.291 < b3(N) < 0.451
			0: 0.201	1: 0.000	Unc: 0.799

	[0.199] R3: 0.096 < b2(N) < 0.202
			0: 0.199	1: 0.000	Unc: 0.801

	[0.197] R36: b8(N) < 0.029
			0: 0.195	1: 0.005	Unc: 0.800

	[0.196] R2: -0.010 < b2(N) < 0.096
			0: 0.193	1: 0.006	Unc: 0.800

	[0.193] R15: 0.201 < b6(N) < 0.350
			0: 0.186	1: 0.013	Unc: 0.801

Most important rules for class 1

	[0.242] R60: b12(N) < 0.403
			0: 0.000	1: 0.242	Unc: 0.758

	[0.241] R63: 0.766 < b12(N) < 0.928
			0: 0.000	1: 0.241	Unc: 0.759

	[0.235] R22: 0.477 < b9(N) < 0.689
			0: 0.000	1: 0.235	Unc: 0.765

	[0.230] R13: -0.135 < b6(N) < 0.052
			0: 0.000	1: 0.230	Unc: 0.770

	[0.200] R18: b9(N) < -0.075
			0: 0.000	1: 0.200	Unc: 0.800

	[0.200] R6: b7(N) < -0.076
			0: 0.001	1: 0.199	Unc: 0.800

	[0.197] R25: -0.077 < b5(N) < -0.024
			0: 0.006	1: 0.194	Unc: 0.800